# Machine Learning SoSe21 Practice Class

Dr. Timo Baumann, Dr. Özge Alaçam, Björn Sygo <br>
Email: baumann@informatik.uni-hamburg.de, alacam@informatik.uni-hamburg.de, 6sygo@informatik.uni-hamburg.de


## Exercise 3
**Description:** Implement gaussian discriminant analysis on the provided images <br>
**Deadline:** Saturday, 15. May 2021, 23:59 <br>
**Working together:** You can work in pairs or triples but no larger teams are allowed. <br>
&emsp;&emsp;&emsp; &emsp; &emsp; &emsp; &emsp; Please adhere to the honor code discussed in class. <br>
&emsp;&emsp;&emsp; &emsp; &emsp; &emsp; &emsp; All members of the team must get involved in understanding and coding the solution.

## Submission: 
**Put your names here**

*Also put high-level comments that should be read before looking at your code and results.*

### Goal

1. The goal of this exercise is to build a classifier for real-life data (images).
2. You will derive features of the images and then use GDA for classification, i.e., you compute the probability of each image being sampled from one of the classes $j$ (in our case $j \in \{\textrm{no_mask}, \textrm{mask}\}$). This probability can be calculated with <br>
$p(y=j|x)= \frac{p(x|y=j)p(y=j)}{p(x|y=j)p(y=j)+p(x|y \neq j)p(y \neq j)}$ <br>
where <br>
$p(x|y=j)=\frac{1}{(2 \pi)^{\frac{n}{2}}|\Sigma|^{\frac{1}{2}}}e^{-\frac{1}{2}(x-\mu_j)^T \Sigma^{-1} (x-\mu_j)}$ <br>
and the prior probability $p(y=j)=1-\Phi$ (depending on the dataset).
3. For later classification, you compute for each class the probability that the image is a sample of it and choose the class with the highest probability.

### Load the images

**Task 1** (15%): Load the images and represent them so that you can work with them.

The dataset contains identically sized images of people with and without facemasks and the goal is to classify if an image contains a person wearing a facemask or not. Note that some images have 3 colors, some also have an alpha channel which you should probably ignore.

When the images are loaded, they are represented as a 32x32x3 matrix. One of the 3 layers of the matrix is for the red (R) value, one for the 
green (G) value and one for the blue (B) value. The image itself is created by taking the values of each of theses 3 matrices at (i,j) to create the 
pixel of the image at spot (i,j). Each of the values can be in range of 0-255.

First, you should load in the images (see zip files). There are two versions of the dataset, a small subsample which contains 40 images for each of the two classes, and the large full dataset (with imbalanced classes). There are multiple libraries for image representation. Try PIL or google around.

In [13]:
from PIL import Image as im
import glob

def load_imgs_to_array(path):
    
    imgs = []
    for imgpath in glob.iglob(path + "*"):
        img=im.open(imgpath)
        image = []
        for row in range(img.height):
            pixels = []
            for column in range(img.width):
                pixels.append(img.getpixel((row, column))[:3])
            #print(row)
            image.append(pixels)
        imgs.append(image)
    return imgs


#mask_path = "subset/mask/"
#nomask_path = "subset/no_mask/"

mask_imgs = load_imgs_to_array("subset/mask/")
nomask_imgs = load_imgs_to_array("subset/no_mask/")

print(len(mask_imgs)) #Anzahl Bilder
#print(len(nomask_imgs))
print(len(mask_imgs[0])) #Anzahl Reihen
print(len(mask_imgs[0][0])) #Anzahl Pixel pro Reihe
print(len(mask_imgs[0][0][0])) #Anzahl Farbkanäle pro Pixel
#mask_imgs[0].show()

40
32
32
3


### Obtain feature vectors

**Task 2** (15%): Build a feature vector and represent each image by its corresponding feature vector.

For Gaussian Discriminant Analysis, the images should be represented as feature vectors. A feature vector consists of different features of the image, for example mean or variance of pixel values, of color channels, number of "blue" pixels, etc. Be creative. The more discriminative your features, the better your classifier will perform.

Your feature vector should contain at least 5 different features. (Note: your code below should also work with more or fewer features.)

In [14]:
image = mask_imgs[0]
pixel_count = len(image) * len(image[0])


def meanValues(image):
    mean_image = [0, 0, 0]
    for row in image:
        for pixel in row:
            for index, chan in enumerate(pixel):
                mean_image[index] += chan


    
    for index, channel in enumerate(mean_image):
        mean_image[index] /= pixel_count
    return mean_image

print(meanValues(image))

def sigmaValues(image):
    sigma_image = [0, 0, 0]
    mean_image = meanValues(image)
    for row in image:
        for pixel in row:
            for index, chan in enumerate(pixel):
                sigma_image[index] += (chan - mean_image[index])**2


    
    for index, channel in enumerate(sigma_image):
        sigma_image[index] = (sigma_image[index] / pixel_count) ** (1/2)
    return sigma_image

print(sigmaValues(image))


def meanValuesAllImages(images):
    new_images = []
    for image in images:
        new_images.append(meanValues(image))
    return new_images

#print(meanValuesAllImages(mask_imgs))







[137.1005859375, 124.8974609375, 120.0791015625]
[53.051554374322315, 54.79423720660013, 54.675468254787816]


In [15]:
## maskfilter mean
maskFilter = [
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2],
[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
[0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8],
[0, 0.5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.5, 0],
[0, 0.5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.5, 0],
[0, 0.5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.5, 0],
[0, 0.5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.5, 0],
[0, 0.5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.5, 0],
[0, 0.5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.5, 0],
[0, 0.5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.5, 0],
[0, 0.5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.5, 0],
[0, 0.5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.5, 0],
[0, 0.5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.5, 0],
[0, 0.5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.5, 0],
[0, 0.5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.5, 0],
[0, 0.5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.5, 0],
[0, 0.5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.5, 0],
[0, 0.5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.5, 0],
[0, 0.5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.5, 0],
[0, 0.5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.5, 0],
[0, 0.5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.5, 0],
[0, 0.5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.5, 0],
[0, 0.5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.5, 0],
[0, 0.5, 0.5, 0.5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.5, 0.5, 0.5, 0],
[0, 0.5, 0.5, 0.5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.5, 0.5, 0.5, 0],
[0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

image = nomask_imgs[3]

def meanValuesFiltered(image, filter):
    mean_image = [0, 0, 0]
    for row in image:
        for i, pixel in enumerate(row):
            for j, chan in enumerate(pixel):
                mean_image[j] += chan*filter[i][j]


    
    for index, channel in enumerate(mean_image):
        mean_image[index] /= pixel_count
    return mean_image

print(meanValuesFiltered(image, maskFilter))

blue_min = [0, 0, 100]
blue_max = [180, 180, 255]


white_min = [128,128,128]
white_max = [255,255,255]

def countColorValues(image, color_min, color_max):
    count = 0
    for row in image:
        for pixel in row:
            is_in_range = True
            for index in range(len(pixel)):
                if not color_min[index] <= pixel[index] <= color_max[index]:
                    is_in_range = False
            count += is_in_range
    return count

print(countColorValues(nomask_imgs[0], blue_min, blue_max))
print(countColorValues(mask_imgs[0], blue_min, blue_max))

print(countColorValues(nomask_imgs[0], white_min, white_max))
print(countColorValues(mask_imgs[0], white_min, white_max))

            


def featureVector(image):
    blue_value = countColorValues(image, blue_min, blue_max)
    white_value = countColorValues(image, white_min, white_max)
    mean = meanValues(image)
    sigma = sigmaValues(image)
    mask_filter = meanValuesFiltered(image, maskFilter)
    return [blue_value, white_value] + mean + sigma + mask_filter

print(featureVector(mask_imgs[0]))



[6.649707031250004, 57.461328124999994, 90.93779296874999]
80
188
451
455
[188, 455, 137.1005859375, 124.8974609375, 120.0791015625, 53.051554374322315, 54.79423720660013, 54.675468254787816, 5.156933593749999, 53.23232421875, 94.83349609375003]


### Initialize your parameters

**Task 3** (5%):  Initialize your parameters for the GDA algorithm.

For the discriminant analysis, you will need to estimate your parameters $\Phi, \mu_0, \mu_1, \Sigma$.

For this, you will need to initialize them first. You can just initalize them with 0, but you should consider their dimensions.

In [16]:
test_feature_vector = featureVector(nomask_imgs[0])

mew_zero = [0 for _ in test_feature_vector]
mew_one = [0 for _ in test_feature_vector]
print(mew_one)

covar_matrix = [[0 for _ in test_feature_vector] for _ in test_feature_vector]
print(covar_matrix)

#phi = len(mask_imgs)/(len(mask_imgs)+(len(nomask_imgs)))
phi = 0
print(phi)


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
0


### Implement Gaussian Discriminant analysis

**Task 4** (35%):  Implement the Gaussian Discriminant Analysis algorithm.

Now you can use the GDA algorithm to find an estimation for the correct parameters to classify the images later.

In [17]:
import numpy as np

def calcPhi(mask_images, nomask_images):
    phi = len(mask_imgs)/(len(mask_imgs)+(len(nomask_imgs)))
    return phi

def calcMew(images, mew):
    count_pictures = len(images)
    m = np.zeros(len(mew))
    for image in images:
        temp_FV = featureVector(image)
        m += temp_FV
        # for index, feature in enumerate(temp_FV):
        #    mew[index] += feature / count_pictures 
    return m / count_pictures

print(calcPhi(mask_imgs, nomask_imgs))
print(calcMew(nomask_imgs, mew_zero))
print(calcMew(mask_imgs, mew_one))

def calcSigma(mask_images, no_mask_images, mew1, mew2):
    # m1 = np.array(mew1)
    # m2 = np.array(mew2)
    return (calcDifference(mask_images, mew1) + calcDifference(no_mask_images, mew2)) / (len(mask_imgs)+len(no_mask_images))

   
def calcDifference(images, mew):
    sigma = np.zeros((mew.shape[0], mew.shape[0]))
    for image in images:
        tempFV = np.array(featureVector(image))
        difference_mask = np.reshape(tempFV - mew, (1, mew.shape[0]))
        # print(difference_mask)
        sigma += difference_mask * difference_mask.T
    return sigma

mu_1 = calcMew(mask_imgs, mew_one)
mu_2 = calcMew(nomask_imgs, mew_zero)
print(calcSigma(mask_imgs, nomask_imgs, mu_1, mu_2))

0.5
[201.25       331.625      146.54331055 120.32631836 106.54619141
  52.98805183  49.78760726  48.12239212   6.09666016  49.49554932
  78.90884766]
[247.725      586.15       159.55598145 155.2298584  152.03708496
  53.99831647  60.39948211  65.48511816   6.26049561  67.33541992
 122.96926025]
[[ 1.40822434e+04 -3.65349500e+03 -1.02186818e+03 -5.28374248e+02
  -3.07383255e+02 -6.44844053e+02 -5.52217144e+02 -4.76976215e+02
  -4.99649460e+01 -2.95420749e+02 -3.79828124e+02]
 [-3.65349500e+03  2.81683309e+04  3.52384237e+03  3.28282589e+03
   3.14517231e+03  2.64176825e+02  3.39351211e+02  3.40200532e+02
   1.24479581e+02  1.39014571e+03  2.37463202e+03]
 [-1.02186818e+03  3.52384237e+03  6.50365863e+02  5.26148653e+02
   4.52834566e+02 -1.53631477e+01  4.27861321e+00  1.28751728e+01
   2.13760840e+01  2.26106036e+02  3.48921730e+02]
 [-5.28374248e+02  3.28282589e+03  5.26148653e+02  4.81151103e+02
   4.37053876e+02 -2.82522895e+01  2.88286595e+00  1.41893624e+01
   1.67494959e+01  2.

### Classify with the Bayes rule
**Task 5** (10%): Use the Bayes rule to check how many images were correctly classified.

Now that you have estimated your parameters, you can use the Bayes rule to classify the images. You then can evaluate how many of the images were correctly classified and try again with different features if the results weren't good enough.

### Cross-validation

**Task 6** (10%): Implement 10-fold cross-validation and report the quality of your results in terms of accuracy and f-score.

You have so far trained and tested your classifier on the same data. This does not tell us much about the true performance on unseen data. 
Instead, you should now randomly split your data into _k_ folds of equal size. you then train your model _k_ times, using all but the _k_'th fold for training and the _k_'th fold for testing.

### Feature importance

**Task 7** (10%): Experiment with the features: how well does the classifier perform with individual features, what is the additional value of the second best feature in addition to the best?

### Report Submission

Prepare a report of your solution as a commented Jupyter notebook (using markdown for your results and comments); include figures and results.
If you must, you can also upload a PDF document with the report annexed with your Python code.

Upload your report file to the Machine Learning Moodle Course page. Please make sure that your submission team corresponds to the team's Moodle group that you're in.